In [44]:
import sys
import gc
import re
import json
import _pickle as p
import pandas as pd
from importlib import reload
from IPython.display import display, HTML

sys.path.append('..')
sys.path.append('../lib')
import stats as S
import tf as T
S = reload(S)
T = reload(T)

pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
display(HTML("<style>.container{width: 100%}</style>"))

In [36]:
lemm = T.lemm

converters_pharma = {
    'ActiveSubstance': str, 
    'PharmaName': str, 
    'AnaloguesText': str, 
    'AnalogsLinks': str, 
    'PharmaLink-href': str, 
    'Lots-href': str,
    'Аптека/Адрес': str,
    'Метро': str,
    'Цена': str
}
pharma = pd.read_csv('initial/pharma.csv', converters=converters_pharma)
pharma = pharma.loc[pharma['PharmacologicAction'].notnull()]
pharma.loc[:, :] = pharma.applymap(str)
pharma.drop(['web-scraper-order', 'web-scraper-start-url', 'Page', 'Page-href', 'PharmaLink', 'AnalogsLinks-href', 'Lots', 'Instruction'], axis=1, inplace=True)
pharma.rename(columns={"PharmacologicAction": "Targ"}, inplace=True)
pharma.loc[:, 'Targ'] = pharma.loc[:, 'Targ'].apply(lambda x: x.replace('Фармакологическое действие', ''))
pharma = pharma.groupby('PharmaName').agg({
    'ActiveSubstance': 'first', 
    'Targ': 'first', 
    'AnaloguesText': "\n".join, 
    'AnalogsLinks': "\n".join, 
    'PharmaLink-href': 'first', 
    'Lots-href': "\n".join,
    'Аптека/Адрес': "\n".join,
    'Метро': "\n".join,
    'Цена': "\n".join,
}).reset_index()
pharma.rename(columns={'PharmaName': 'Name'}, inplace=True)
pharma.loc[:, 'As'] = pharma.loc[:, 'ActiveSubstance'].apply(lambda x: lemm(x))

In [37]:
def extAction(text, pattern=re.compile("Фармакологические свойства[^\n]+")):
    try: 
        a = pattern.search(text)[0]
        try: return a.split(':', 1)[-1]
        except: return a
    except: return ""
        
def extPresc(text, pattern=re.compile("Показания к применению[^\n]+")):
    return extAction(text, pattern=pattern)

def extContr(text, pattern=re.compile("Противопоказания[^\n]+")):
    return extAction(text, pattern=pattern)

def extSubst(text, pattern=re.compile("Международное непатентованное название[^\n]+")):
    e = extAction(text, pattern=pattern)
#     print(e)
    return e.split('Найти цену', 1)[0]


# def extAnalogs
pharma2 = pd.read_csv('initial/pharma2.csv')
pharma2.drop(['web-scraper-order', 'web-scraper-start-url', 'LetterLink', 'LetterLink-href'], axis=1, inplace=True)
display(pharma2.shape)
pharma2.loc[:, 'Action'] = pharma2.loc[:, 'Description'].apply(extAction)
pharma2.loc[:, 'Presc'] = pharma2.loc[:, 'Description'].apply(extPresc)
pharma2.loc[:, 'Contr'] = pharma2.loc[:, 'Description'].apply(extContr)
pharma2.loc[:, 'Subst'] = pharma2.loc[:, 'Description'].apply(extSubst)
pharma2.drop(['Description', 'Action'], axis=1, inplace=True)
pharma2.rename(columns={'PharmaLink': 'Name', 'Presc': 'Targ', 'PharmaLink-href': 'Link'}, inplace=True)
# print(pharma2.shape, pharma2[pharma2['Action'] == ""].shape)
# pharma2.head(10)

(3483, 3)

In [38]:
pattern = re.compile('[^\w\d\s]+')
sdict = pd.read_csv('initial/def.csv')
sdict.drop(['web-scraper-order', 'web-scraper-start-url', 'Pages', 'Pages-href'], axis=1, inplace=True)
sdict.loc[:, 'Name'] = sdict.loc[:, 'Definitions'].apply(lambda x: pattern.split(x, 1)[0])
sdict.loc[:, 'Definitions'] = sdict.loc[:, 'Definitions'].apply(lambda x: pattern.split(x, 1)[1])
sdict.rename(columns={"Definitions": "Targ"}, inplace=True)
# sdict.head()

In [39]:
pdict = pd.read_csv('initial/prof.csv')
pdict.drop(['web-scraper-order', 'web-scraper-start-url', 'ProfLinks-href'], axis=1, inplace=True)
pdict.drop([1, 3, 4, 5, 11, 16, 23, 26, 27, 33], axis=0, inplace=True)
pdict.rename(columns={"DescriptionText": "Targ"}, inplace=True)
pdict.rename(columns={"ProfLinks": "Name"}, inplace=True)

In [45]:
sdict2 = pd.read_csv('initial/def2.csv', converters={'Description': str})
sdict2.drop(['web-scraper-order', 'web-scraper-start-url', 'Disease-href'], axis=1, inplace=True)
sdict2.rename(columns={"Description": "Targ"}, inplace=True)
sdict2 = sdict2.groupby('Disease').agg({'Targ': 'sum'}).reset_index(drop=False)
sdict2.rename(columns={'Disease': 'Name'}, inplace=True)
sdict2.head()

Name  \
0    Абсцесс головного мозга   
1            Абсцесс легкого   
2  Адреногенитальный синдром   
3                Акромегалия   
4                Актиномикоз   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [41]:
# Ph = T.prepare_dataset(pharma, 'pharma', max_len=None)
# Sd = T.prepare_dataset(sdict, 'def', max_len=None)
Sd2 = T.prepare_dataset(sdict2, 'def2', max_len=None)
Pd = T.prepare_dataset(pdict, 'prof', max_len=None)
Pd2 = T.prepare_dataset(pharma2, 'pharma2', max_len=None)

In [42]:
# display(Ph.head())
# display(Sd.head())
display(Sd2.head())
display(Pd.head())
display(Pd2.head())

Name  \
0    Абсцесс головного мозга   
1            Абсцесс легкого   
2  Адреногенитальный синдром   
3                Акромегалия   
4                Актиномикоз   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

,Name,Targ,Tf-Idf,KwFilt
0,Диетолог,"Дието́лог — специалист в области диетологии, соответствующий академическим и профессиональным требованиям. Врач-диетолог является дипломированным врачом, закончившим медицинский вуз и получившим затем дополнительное образование по диетологии, подтвержденное соответствующим дипломом. Является экспертом в области питания. Врачи-диетологи занимаются лечением ожирения, а также дают рекомендации пациентам по вопросам правильного питания, ведения здорового образа жизни, либо ставят цели для достижения определённых результатов, связанных со здоровьем.","{'диет': 3.2188758248682006, 'лог': 0.3218875824868201, 'специалист': 0.041415606803781584, 'област': 0.18185223940184106, 'диетолог': 3.2188758248682006, 'соответств': 1.2628643221541278, 'академическ': 3.2188758248682006, 'профессиональн': 1.2628643221541278, 'требован': 3.2188758248682006, 'врач': 0.022686028283057925, 'явля': 0.3218875824868201, 'дипломирова': 3.2188758248682006, 'заканчива': 3.2188758248682006, 'медицинск': 0.041415606803781584, 'вуз': 3.2188758248682006, 'получа': 0.18185223940184106, 'зат': 3.2188758248682006, 'дополнительн': 3.2188758248682006, 'образован': 0.041415606803781584, 'подтвержда': 3.2188758248682006, 'дипл': 1.2628643221541278, 'эксперт': 1.2628643221541278, 'питан': 3.2188758248682006, 'занима': 0.2378527259400243, 'лечен': 0.18185223940184106, 'ожирен': 3.2188758248682006, 'такж': 1.2628643221541278, 'дава': 3.2188758248682006, 'рекомендац': 3.2188758248682006, 'пациент': 0.2378527259400243, 'вопрос': 0.7067545120666969, 'правильн': 3.2188758248682006, 'веден': 3.2188758248682006, 'здоров': 1.2628643221541278, 'образ': 3.2188758248682006, 'жизн': 1.2628643221541278, 'либ': 1.2628643221541278, 'став': 3.2188758248682006, 'цел': 3.2188758248682006, 'достижен': 3.2188758248682006, 'определен': 1.2628643221541278, 'результат': 3.2188758248682006, 'связыва': 1.2628643221541278}","{'диет': 3.2188758248682006, 'лог': 0.3218875824868201, 'специалист': 0.041415606803781584, 'област': 0.18185223940184106, 'диетолог': 3.2188758248682006, 'соответств': 1.2628643221541278, 'академическ': 3.2188758248682006, 'профессиональн': 1.2628643221541278, 'требован': 3.2188758248682006, 'явля': 0.3218875824868201, 'дипломирова': 3.2188758248682006, 'заканчива': 3.2188758248682006, 'медицинск': 0.041415606803781584, 'вуз': 3.2188758248682006, 'получа': 0.18185223940184106, 'зат': 3.2188758248682006, 'дополнительн': 3.2188758248682006, 'образован': 0.041415606803781584, 'подтвержда': 3.2188758248682006, 'дипл': 1.2628643221541278, 'эксперт': 1.2628643221541278, 'питан': 3.2188758248682006, 'занима': 0.2378527259400243, 'лечен': 0.18185223940184106, 'ожирен': 3.2188758248682006, 'такж': 1.2628643221541278, 'дава': 3.2188758248682006, 'рекомендац': 3.2188758248682006, 'пациент': 0.2378527259400243, 'вопрос': 0.7067545120666969, 'правильн': 3.2188758248682006, 'веден': 3.2188758248682006, 'здоров': 1.2628643221541278, 'образ': 3.2188758248682006, 'жизн': 1.2628643221541278, 'либ': 1.2628643221541278, 'став': 3.2188758248682006, 'цел': 3.2188758248682006, 'достижен': 3.2188758248682006, 'определен': 1.2628643221541278, 'результат': 3.2188758248682006, 'связыва': 1.2628643221541278}"
2,Акушерка,"Акуше́рка (фр. accoucher — помогать при родах[1]) — средний медицинский работник, специалист в акушерстве, оказывающий помощь беременным и роженицам (во время родов). В России для работы акушеркой необходимо получить среднее специальное образование по специальности «Медико-акушерское дело». Квалификации: «акушер(ка)», «фельдшер-акушер». Не следует путать с врачом акушер-гинекологом (высшее медицинское образование).","{'акуш': 1.2628643221541278, 'рка': 1.2628643221541278, 'фр': 3.2188758248682006, 'accoucher': 3.2188758248682006, 'помога': 3.2188758248682006, 'род': 0.7067545120666969, '1': 0.07463459564271185, 'средн': 0.45814536593707755, 'медицинск': 0.041415606803781584, 'работник': 1.2628643221541278, 'специалист': 0.041415606803781584, 'акушер

,Name,Link,Targ,Contr,Subst,Tf-Idf,KwFilt
0,Эскузан,http://drugs.thead.ru/Eskuzan,"Хроническая венозная недостаточность варикозного и посттромботического генеза и ее осложнения, в т.ч. заболевания, связанные с функциональным нарушением кровоснабжения:",— повышенная чувствительность к компонентам препарата.,,"{'хроническ': 0.0036466395942660926, 'венозн': 0.197470937819912, 'недостаточн': 0.015639254831611844, 'варикозн': 0.27427422585260847, 'посттромботическ': 2.3523456605659634, 'генез': 0.016961166337090054, 'е': 0.36317327128425275, 'осложнен': 0.07542392882060964, 'т': 0.0025574035457321138, 'ч': 0.002634051349962679, 'заболеван': 0.0017493674034666617, 'связыва': 0.03216220410409057, 'функциональн': 0.11166924540630163, 'нарушен': 0.011145876445933374, 'кровоснабжен': 2.3523456605659634}","{'венозн': 0.197470937819912, 'варикозн': 0.27427422585260847, 'посттромботическ': 2.3523456605659634, 'е': 0.36317327128425275, 'осложнен': 0.07542392882060964, 'связыва': 0.03216220410409057, 'функциональн': 0.11166924540630163, 'кровоснабжен': 2.3523456605659634}"
1,Равел СР,http://drugs.thead.ru/Ravel_SR,— артериальная гипертензия.,— почечная недостаточность тяжелой степени (КК < 30 мл/мин);,Индапамид,"{'артериальн': 0.01670711078693211, 'гипертенз': 0.016337857674586603}","{'артериальн': 0.01670711078693211, 'гипертенз': 0.016337857674586603}"
2,Строфантин К,http://drugs.thead.ru/Strofantin_K,"— в составе комплексной терапии острой и хронической сердечной недостаточности II функционального класса (при наличии клинических проявлений), III-IV функционального класса по классификации NYHA;",— гликозидная интоксикация;,Строфантин-к,"{'соста': 0.008655169410521026, 'комплексн': 0.012953540661573665, 'терап': 0.002466166233539272, 'остр': 0.005875013425255062, 'хроническ': 0.0036466395942660926, 'сердечн': 0.043682329575008054, 'недостаточн': 0.015639254831611844, 'ii': 0.1523116795445437, 'функциональн': 0.11166924540630163, 'класс': 2.3523456605659634, 'налич': 0.09935135537381226, 'клиническ': 0.25799584984060053, 'проявлен': 0.24338699203059896, 'iii': 0.1411861123908946, 'iv': 0.8871055887586696, 'классификац': 0.47256188504816676, 'nyha': 3.731251044903028}","{'сердечн': 0.043682329575008054, 'ii': 0.1523116795445437, 'функциональн': 0.11166924540630163, 'класс': 2.3523456605659634, 'налич': 0.09935135537381226, 'клиническ': 0.25799584984060053, 'проявлен': 0.24338699203059896, 'iii': 0.1411861123908946, 'iv': 0.8871055887586696, 'классификац': 0.47256188504816676, 'nyha': 3.731251044903028}"
3,Тирозол,http://drugs.thead.ru/Tirozol,— тиреотоксикоз;,— повышенная чувствительность к компонентам препарата и к производным тиомочевины;,Тиамазол,{'тиреотоксикоз': 0.7595259660857706},{'тиреотоксикоз': 0.7595259660857706}
4,Шиповника плоды низковитаминные (порошок),http://drugs.thead.ru/Shipovnika_plody_nizkovitaminnye_poroshok,Для профилактики и лечения гиповитаминозов C и P.,"Желчнокаменная болезнь, повышенная чувствительность к растению.",Шиповника плоды низковитаминные,"{'профилактик': 0.0037226186865969573, 'лечен': 0.0023244629825060472, 'гиповитаминоз': 0.1411861123908946, 'c': 0.3364412842578888, 'p': 0.8871055887586696}","{'гиповитаминоз': 0.1411861123908946, 'c': 0.3364412842578888, 'p': 0.8871055887586696}"


In [43]:
gc.collect()

NameError: name 'gc' is not defined

In [135]:
pharma.to_csv('tmp', index=False)